In [51]:
import numpy as np
import pandas as pd

#РАСПРЕД ПО СПИСКУ МАГОВ
#список складов для распреда
all_llist = [3515,1074,3403,3684,347,4759,3583,4774,4856,3918,3643,3361,3264,4737,3635,4772,3294,3224,3689,3180,3624,3785,3013,3078,3908,3519,3955,4517,3205,3698,570,3564,3697,3144,3841,107,3477,4314,4422,3117,3690,4724,4397,3680,3799,240,4420,3368,3299,4425,3418,4333,4641,3038,4423,3281,3414,356,96,3421,4428,3494,4310,4668,4822,3156,374,533,4806,3000,3509,4747,3293,4542,3087,3162,4579,4633,3816,4841,4707,4587,3430,4695,302,4723,4525,3146,3651,3644,4574,10,4618,3112,3011,4504,3260,3354,4798,3220,788,3338,9,4451,3638,4399,4784,4753,4805,3159,3742,4760,3810,789,3621,4727,3211,3464,3450,3536,3622,4586,4365,4513,3043,779,3276,3484,4666,4403,3506,185,4660,3894,4644,4789,3269,4448,4507,99,4726,4358,4352,3954,4762,4729,525,4597,4665,3649,4601,3466,4465,3976,3582,561,3650,4363,4544,955,3927,3429,4610,3407,4309,753,3271,3085,3614,4740,551,544,4580,3951,4394,3216,4441,4620,3532,3507,4522,3616,3143,117,4573,4453,4648,3596,4510,3617,781,3487,4616,3847,3315,3701,759,3630,4509,3626,4460,3893,3189,3712,4650,4716,4452,3598,581,564,4547,752,3925,4449,3196,3574,270,4427,3822,3831,4336,3516,4623,4512,384,3986,3696,4454,4368,4625,4381,3370,3765]


prih1 = pd.read_csv(r'C:\Users\Dotsenko.Semen\Documents\ItemMoves.csv', delimiter=';', encoding='cp1251',usecols=list(range(0, 48)))
del prih1['№ Поставщика']
#ИЛИ
#prih1 = pd.read_excel(r'C:\рушники\123.xlsx')

prih1 = prih1.sort_values(by='Товар')

# словарь тов - количество и список товов
tov_list = prih1.Товар.unique().tolist()
tov_dict = prih1.Товар.value_counts()[prih1.Товар.unique()].to_dict()

# остатки
#stock = pd.read_excel(r'C:\Остатки\обручи.xlsx', usecols=['КодСклада','ТоварНо'])
stock = pd.read_excel(r'C:\Остатки\остатки.xlsx', usecols=['КодСклада','ТоварНо'])
stock = stock[stock['КодСклада'].isin(all_llist) & stock['ТоварНо'].isin(tov_list)]

# таблица остатки товов по складам
pivot = pd.pivot_table(stock,index='КодСклада', columns='ТоварНо', aggfunc=len, fill_value=0).reset_index()
pivot = pd.DataFrame({'КодСклада':all_llist}).merge(pivot, how='left')

# добавим в таблицу колонки с нулями по товам, которых не хватает
add_cols =  [x for x in tov_list if x not in pivot.columns.to_list()]
add_zeroes = [list(np.zeros(len(all_llist), dtype='int'))] * len(add_cols)

pivot = pivot.reindex(columns=pivot.columns.tolist() + add_cols)

pivot.fillna(0, inplace=True)
pivot = pivot.astype('int')
pivot = pivot[['КодСклада'] + tov_list]

# Если количество по тову в прихе больше чем в списке то к списку добавить нули, если нет то обрезать
sklad_list = [
    pivot[pivot[x] == 0]['КодСклада'].tolist() + 
    list(np.zeros((tov_dict[x] - len(pivot[pivot[x] == 0]['КодСклада'].tolist())), dtype='int')) 
    if 
    tov_dict[x] > len(pivot[pivot[x] == 0]['КодСклада'].tolist()) 
    else pivot[pivot[x] == 0]['КодСклада'].tolist()[:tov_dict[x]] 
    for x in tov_list
]

sklad_list = [item for sublist in sklad_list for item in sublist]

prih1['Период реализации'] = sklad_list

#prih1 = prih1[prih1['Период реализации'] > 0]

# Сохраняем в ексель на лист "движение товара"

import random
name = 'C:/рушники/' + str(random.randrange(5000,10000)) + '.xlsx'
prih1.to_excel(name, sheet_name='Движение товара', index=False)

In [48]:
prih1[['Период реализации', 'Склад']].groupby(by='Период реализации').count().sort_values(by='Склад', ascending=False)


,Склад
Период реализации,
396,17
4574,16
3061,15
354,15
569,15
...,...
526,3
3156,3
565,2
